<h1>Olympic History Analysis</h1>

In this notebook, we had the goal to analysis the [<font color='orange'>120 years of Olympic History</font>](https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results) dataset in order to produce any meaningful insight.


<h2><font color='#0E2545'>1. Load the Olympic History dataset</font></h2>

In order to obtain and use the dataset for this analysis, follow this steps:
1. Download it from the following url:
https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results

2. Unzip data set package and move csv file to the data_source folder



In [1]:
#Import modules
%matplotlib inline
import numpy as np
from sklearn.preprocessing import Imputer
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#Load data sets
athlete_events_df = pd.read_csv("data_source/athlete_events.csv")
noc_regions_df = pd.read_csv("data_source/noc_regions.csv")

FileNotFoundError: File b'data_source/athlete_events.csv' does not exist

<h2><font color='#0E2545'>2. Prepare Data</font></h2>


<h3><font color = '#082E86'> 2.A Explore Data </font> <font></h3>

In [ ]:
athlete_events_df.head(5)

In [ ]:
athlete_events_df.shape

In [ ]:
athlete_events_df.describe()

In [ ]:
noc_regions_df.head(5)

In [ ]:
#Join both dataframes
athlete_events_df = athlete_events_df.merge(noc_regions_df, how = 'inner', on=["NOC"])

#Setting column names to upper case
athlete_events_df.columns = athlete_events_df.columns.map(lambda c: c.upper())
athlete_events_df.head(5)

**Hypothesis**
1. Athletes are taller as time passes through
2. The average of age is not the same for every sport

**1. Athletes are taller as time passes through**

In [ ]:
male_athletes_df = athlete_events_df[athlete_events_df.SEX == "M"]
male_athletes_average_heights_df = male_athletes_df.groupby("YEAR").agg({"HEIGHT":"mean"}).reset_index()
female_athletes_df = athlete_events_df[athlete_events_df.SEX == "F"]
female_athletes_average_heights_df = female_athletes_df.groupby("YEAR").agg({"HEIGHT":"mean"}).reset_index()


In [ ]:
plt.figure(1)
plt.subplot(221)
plt.scatter(x = male_athletes_average_heights_df.YEAR, y = male_athletes_average_heights_df.HEIGHT)

plt.subplot(222)
plt.scatter(x = female_athletes_average_heights_df.YEAR, y = female_athletes_average_heights_df.HEIGHT)

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                    wspace=0.35)
plt.show()

Indeed, since 1960 (data is more stable), it's observed that height average has increased for both genders.

**2. The average of age is not the same for every sport**

In [ ]:
average_age_per_sport_df = athlete_events_df.groupby("SPORT").agg({"AGE":[np.mean,np.std,np.min,np.max]}).reset_index()
average_age_per_sport_df.head(10)

In [ ]:
average_age_per_sport_df.describe()

In [ ]:
plt.hist(x=average_age_per_sport_df["AGE"]["mean"].values,bins = 5)
plt.title("Average Age per Sport Distribution")
plt.show()

Each average doesn't varies too much from each other, usually the range of ages is between 18 - 32 years. Older athletes are present, but aren't a majority.

In [ ]:
athlete_events_df.info()

<h3><font color = '#082E86'> 2.B Data Cleaning </font> <font></h3>

As we observed, the following columns contain missing values:
- Age
- Height
- Weight
- Notes

For each column we applied different approaches in order to fix each missing value.

In [ ]:
#We aren't interested on sports that don't have enough records
#At least for 30% of all years, and at least 10% of all teams
minimum = athlete_events_df.YEAR.unique().size * 0.3 * athlete_events_df.TEAM.unique().size*0.1
records_per_sport = athlete_events_df.groupby("SPORT").count().reset_index()[["SPORT","ID"]]
invalid_sports = records_per_sport[records_per_sport["ID"] < minimum].SPORT.values
athlete_events_df = athlete_events_df[~athlete_events_df.SPORT.isin(invalid_sports)]
athlete_events_df.shape

In [ ]:
minimum

In [ ]:
#We don't need the Notes column, so lets remove it
athlete_events_df = athlete_events_df.drop(columns=["NOTES"])
athlete_events_df.head()

In [ ]:
# If an athlete doesn't have as value for medal, it means he/she didn't won any
# Since age is not dependant to the sport, we may just use the overall mean 
athlete_events_df = athlete_events_df.fillna({"MEDAL": 'No Medal', "AGE": np.round(athlete_events_df["AGE"].mean())})
athlete_events_df.head(5)

In [ ]:
# We consider that Height and Weight vary per Sport, Region and Year, so we decided to create an imputer
#that treats this

def clean_columns_by_imputer(df, groupby, missing_columns, stategy = 'mean'):
    missing_columns_x_mean = {}
    global_means = df[missing_columns].mean()
    for mc in missing_columns:
        missing_columns_x_mean[mc] = global_means[mc]
    df = df.groupby(groupby)
    for name, group in df:
        if group[group[missing_columns].isna()].shape == group.shape:
            group = group.fillna(missing_columns_x_mean)
            yield group
        imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
        imputer = imputer.fit(group[missing_columns].values)
        group[missing_columns] = imputer.transform(group[missing_columns].values)
        yield group

athlete_events_df_cp = athlete_events_df.copy().reset_index()
test = athlete_events_df.copy()
#athlete_events_df_cp.groupby(["SPORT"]).describe().head()
for group in clean_columns_by_imputer(athlete_events_df_cp,["SPORT","YEAR","SEX"], ["HEIGHT","WEIGHT"]):
    test.loc[group["index"].values,"HEIGHT"] = group.loc[:,"HEIGHT"].values
    


In [ ]:
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)


In [ ]:
athlete_events_df["BMI"] = np.round((athlete_events_df["WEIGHT"])/(athlete_events_df["HEIGHT"]/100)**2)
athlete_events_df["BMI_CLASSIFICATION"] = athlete_events_df["BMI"].map(lambda bmi: "Underweight" if bmi < 18.5 else
                                                                      "Normal" if  18.5 <= bmi <= 24.9 else
                                                                      "Overweight" if 25 <= bmi <= 29.9 else
                                                                      "Obese")

In [ ]:
athlete_events_df